In [2]:
from redcap import Project
import csv
import datetime
import pandas as pd

#REDCap API configuration
api_url = 'https://ruff.mrc.gm:8443/redcap/api/'
api_key = '63A451E2DA73C4D6A4C64A17B36B9565'
project = Project(api_url, api_key)

In [3]:
#extract study termination data
supplementation_data = project.export_records(forms=['infant_supplementation'])

#convert data to Dataframe.
supp_data=pd.DataFrame(supplementation_data)

In [4]:
# Specify the fields you want to extract base on logics
df_suppData=supp_data[['con_participantid_q1','sup_infantid','redcap_event_name','redcap_repeat_instrument','redcap_repeat_instance','sup_vdate','sup_completedby']]
df_suppData=pd.DataFrame(df_suppData)

#st_date as a date data type
df_suppData['sup_vdate'] = pd.to_datetime(df_suppData['sup_vdate'], errors='coerce')


df_supp = df_suppData[
    (df_suppData['redcap_event_name'] == 'daily_supplementat_arm_2') &
    (df_suppData['con_participantid_q1'].eq(df_suppData['con_participantid_q1'].shift(-1))) &
    (((df_suppData['sup_vdate'].shift(-1) - df_suppData['sup_vdate']).dt.days < 1) | ((df_suppData['sup_vdate'].shift(-1) - df_suppData['sup_vdate']).dt.days > 1))
]


In [5]:
#INfant data frame
df_supp = pd.DataFrame({
    'ID': df_suppData['sup_infantid'].shift(-1)+"-"+(df_supp['redcap_repeat_instance'] + 1).astype(str)+"-"+(pd.to_datetime(df_suppData['sup_vdate'].shift(-1))).astype(str),
    'con_participantid_q1': df_suppData['con_participantid_q1'].shift(-1),
    'Infantid': df_suppData['sup_infantid'].shift(-1),
    'Instance number': df_suppData['redcap_repeat_instance'].shift(-1),
    'Supplementation date': pd.to_datetime(df_suppData['sup_vdate'].shift(-1)),
    'Instrument': 'Infant Supplementation',
    'Completed by' : df_suppData['sup_completedby'].shift(-1),
    'Description':  'This date difference between instance ' + df_supp['redcap_repeat_instance'].astype(str) + ' and instance ' + (df_supp['redcap_repeat_instance'] + 1).astype(str) + ' is less than 1 or greater than 4.'

})

In [6]:
## EXCLUDE ALL RESOLVED QUERIES
#Read from the ones that are resolved: "RESOLVED_QUERIES.csv"
df_resolvedQ = pd.read_csv('RESOLVED_QUERIES.csv')

# Merge the dataframes on 'ID'
con_merged_df = pd.merge(df_supp, df_resolvedQ, on= 'ID', how='inner')

# Filter out the common ID
df_supp= df_supp[~df_supp['ID'].isin(con_merged_df['ID'])]

In [7]:
#Final Infant Query data frame
df_supp = pd.DataFrame({
    'ID': df_supp['ID'],
    'con_participantid_q1': df_supp['con_participantid_q1'],
    'Infantid': df_supp['Infantid'],
    'Instance number': df_supp['Instance number'],
    'Supplementation date': df_supp['Supplementation date'],
    'Instrument': df_supp['Instrument'],
    'Completed by' : df_supp['Completed by'],
    'Description':  df_supp['Description']

})

In [8]:
df_supp=df_supp[
    (df_supp['Description'].notnull())
]

In [9]:
#Save the filtered DataFrame to a CSV file
df_supp.to_csv('InfSupplementQuery.csv', index=False)